In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
import seaborn as sns
from hepmc.core.densities.qcd import ee_qq_ng, export_hepmc, import_hepmc
from hepmc.core.densities.nice import Nice
from hepmc.core.phase_space.rambo import RamboOnDiet
from hepmc.core.phase_space.mapping import MappedDensity
from hepmc.core.integration.importance import ImportanceMC
from hepmc.core.sampling import Sample, AcceptRejectSampler

## Plot the importance distribution used by Sherpa

In [ ]:
sherpa_weighted_sample = import_hepmc('../samples/qcd/2-5/sherpa_weighted.hepmc')
sherpa_weighted_sample = Sample(data=rambo_mapping.map_inverse(sherpa_weighted_sample.data), weights=sherpa_weighted_sample.weights)

In [ ]:
data = pd.DataFrame(sherpa_weighted_sample.data)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

## Load the training sample and plot it

In [ ]:
eeqq3g = ee_qq_ng(3, 100., 5., .3)
rambo_mapping = RamboOnDiet(100., 5)
mapped = MappedDensity(eeqq3g, rambo_mapping)
training_sample = import_hepmc('../samples/qcd/2-5/training.hepmc')
training_sample = Sample(data=rambo_mapping.map_inverse(training_sample.data), weights=training_sample.weights)

In [ ]:
data = pd.DataFrame(training_sample.data)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

## Train a NICE network based on the training data

In [ ]:
%time nice = Nice(training_sample, train_iters=10000, num_bijectors=5)

### Plot the learned distribution

In [ ]:
sample = nice.rvs(100000)

In [ ]:
data = pd.DataFrame(sample)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15, range=[[0, 1], [0, 1]])

## Use importance sampling to determine the maximum weight

In [ ]:
importance_sampler = ImportanceMC(mapped, nice)
%time nice_sample = importance_sampler.sample(10000)

In [ ]:
bound = nice_sample.weights.max()
print('Maximum weight:', bound)

In [ ]:
exp_eff = nice_sample.weights.mean()/bound
print('Expected unweighting efficiency:', exp_eff)

### Export the weighted sample

In [ ]:
mapped_sample = Sample(data=rambo_mapping.map(nice_sample.data), weights=nice_sample.weights)
export_hepmc(100., mapped_sample, "../samples/qcd/2-5/realnvp_weighted.hepmc")

## Use Acceptance-Rejection method to generate an unweighted sample

In [ ]:
sampler = AcceptRejectSampler(target=mapped, sampling_dist=nice, bound=bound)

In [ ]:
sample = sampler.sample(100, batch_size=50000)

### Export the unweighted sample

In [ ]:
mapped_sample = Sample(data=rambo_mapping.map(sample.data), weights=sample.weights)
export_hepmc(100., mapped_sample, "../samples/qcd/2-5/realnvp.hepmc")